<a href="https://colab.research.google.com/github/Ranjankumar666/text_generator_rnn/blob/main/Text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, LSTM, Dense,  Embedding, TextVectorization
import string
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
!wget https://www.gutenberg.org/files/1400/1400-0.txt

--2022-06-07 01:50:15--  https://www.gutenberg.org/files/1400/1400-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1058019 (1.0M) [text/plain]
Saving to: ‘1400-0.txt’

1400-0.txt          100%[===================>]   1.01M  4.54MB/s    in 0.2s    

2022-06-07 01:50:15 (4.54 MB/s) - ‘1400-0.txt’ saved [1058019/1058019]



In [3]:
data = None

with open('1400-0.txt') as txt:
  data = txt.read()


data

'\ufeffThe Project Gutenberg eBook of Great Expectations, by Charles Dickens\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: Great Expectations\n\nAuthor: Charles Dickens\n\nRelease Date: July, 1998 [eBook #1400]\n[Most recently updated: April 27, 2020]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\nProduced by: An Anonymous Volunteer and David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK GREAT EXPECTATIONS ***\n\n[Illustration]\n\n\n\n\nGreat Expectations\n\n[1867 Edition]\n\nby Charles Dickens\n\n\nContents\n\n Chapter I.\n Chapter II.\n Chapter III.\n Chapter I

In [4]:
data = data.split('\n')

In [5]:
data  = data[107:]

In [6]:
data = " ".join(data)

In [7]:
def clean_text(doc):
  # whitespace separated
  tokens = doc.split()
  #No punctuation
  table = str.maketrans('', '',  string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [ word for word in tokens if word.isalpha()]
  tokens = [ word.lower() for word in tokens ]

  return tokens

In [8]:
tokens = clean_text(data)
tokens[:40]

['my',
 'family',
 'name',
 'being',
 'pirrip',
 'and',
 'my',
 'christian',
 'name',
 'philip',
 'my',
 'infant',
 'tongue',
 'could',
 'make',
 'of',
 'both',
 'names',
 'nothing',
 'longer',
 'or',
 'more',
 'explicit',
 'than',
 'pip',
 'so',
 'i',
 'called',
 'myself',
 'pip',
 'and',
 'came',
 'to',
 'be',
 'called',
 'pip',
 'i',
 'give',
 'pirrip',
 'as']

In [9]:
len(tokens)

176433

In [10]:
len(set(tokens))

11120

In [11]:
seed = 26
lines = []

for i in range(len(tokens)):
  seq = tokens[i:i+seed]
  lines.append(' '.join(seq))

  if i > 100000:
    break

In [13]:
vectorizer = TextVectorization(output_mode='int')
vectorizer.adapt(lines)

sequences = vectorizer(lines)

In [14]:
X , y = sequences[:, :-1], sequences[:, -1] 

In [15]:
X[0]

<tf.Tensor: shape=(25,), dtype=int64, numpy=
array([  12,  369,  237,   93, 2166,    3,   12, 1269,  237, 2165,   12,
       1502, 1501,   59,  143,    6,  208, 1747,  146,  896,   47,   56,
       8476,   78,  153])>

In [16]:
y[0]

<tf.Tensor: shape=(), dtype=int64, numpy=39>

In [17]:
vocab_size = len(vectorizer.get_vocabulary())

In [18]:
y = to_categorical(y, num_classes=vocab_size)

In [23]:
seq_length = X[0].shape[0]
seq_length

25

In [27]:
model = keras.models.Sequential([
  Embedding(input_dim=vocab_size, output_dim=25),
  LSTM(128, return_sequences=True),
  LSTM(128),
  Dense(64, activation='elu'),
  Dense(vocab_size, activation='softmax')
])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 25)          211925    
                                                                 
 lstm (LSTM)                 (None, None, 128)         78848     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 8477)              551005    
                                                                 
Total params: 981,618
Trainable params: 981,618
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='Adam', loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [30]:
model.fit(X, y , batch_size=32, epochs=20)

Epoch 1/20
3126/3126 [==============================] - 36s 9ms/step - loss: 6.3646 - accuracy: 0.0557
Epoch 2/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.8486 - accuracy: 0.0905
Epoch 3/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.6106 - accuracy: 0.1113
Epoch 4/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.4834 - accuracy: 0.1190
Epoch 5/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.3253 - accuracy: 0.1292
Epoch 6/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.2038 - accuracy: 0.1361
Epoch 7/20
3126/3126 [==============================] - 29s 9ms/step - loss: 5.0998 - accuracy: 0.1418
Epoch 8/20
3126/3126 [==============================] - 29s 9ms/step - loss: 4.9989 - accuracy: 0.1459
Epoch 9/20
3126/3126 [==============================] - 29s 9ms/step - loss: 4.9071 - accuracy: 0.1494
Epoch 10/20
3126/3126 [==============================] - 29s 9ms/step - l

In [31]:
def generate_text_seq(model,vectorizer, text_seq_length, seed_text, n_words):
  text  = []
  for _ in range(n_words):
    encoded = vectorizer([seed_text])
    encoded = pad_sequences(encoded, maxlen=text_seq_length, truncating='pre')

    y_predict = model.predict(encoded).argmax(axis=1)[0]

    predicted_word = vectorizer.get_vocabulary()[y_predict]
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)

  return ' '.join(text)

In [45]:
generate_text_seq(model, vectorizer, seq_length, "I will not eat", 3)

'that i was'